In [1]:
number_of_shots = 5

In [2]:
experiment_name = "2023.12.06-05_Llama2_13B_API_5-few-shot_SEMANTIC_SIMILARITY_WITH_DIFFERENT_CLASSES"

In [3]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-13b-chat-hf"
# model_name = "meta-llama/Llama-2-70b-chat-hf"

In [4]:
!pip install -q openai --progress-bar off

In [5]:
!pip install -q langchain --progress-bar off

In [6]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [7]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [8]:
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

In [9]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [10]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data_but_with_different_classes.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [11]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN


In [12]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx")

In [13]:
data_sel = data #.iloc[54:] #.sample(2)

In [14]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 0
# cnt = 54

results = []

for index, row in data_sel.iterrows():

  file = open(log_filename, 'a+', encoding='utf-8')
    
  #print("\n\n##################################################")
  file.write("\n\n##################################################\n")

  #print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')

  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  few_shots = ""

  for train_data_id in few_shots_list:

    # print("##################################################")
    # print(train_data_id)

    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

    # print(train_row)

    # print(train_row.text.values[0])
    # print(train_row.sdg.values[0])

    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

  #print(few_shots)
  file.write(few_shots)
  file.write('\n')

  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
"""+row["text"]+"""
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""



  # print(template)

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
  query = template

  body = {
    "model": model_name,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": query}],
    "temperature": 0.0
  }


  try:
      result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
      print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'].replace("\n"," - "))
      file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+str(result['choices'][0]['message']['content'].replace("\n"," - "))+"\n")
  except Exception as e:
#       print("#############################################################################")
#       print(e, result)
      print(cnt, text_id, sdg_true, "ERROR:"+str(e)+str(result))
      file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR:"+str(e)+str(result)+"\n")
#       print("#############################################################################")
  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    
  result_extended = (cnt, text_id, sdg_true, result)

  #print(result_extended)

  with open(output_datapath+f'/sdg_classification_Llama2_13B_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  cnt+=1
  file.close()

  # if cnt==5:
  #   break

0 6668 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 11--- -  - The text is about a subway derailment in Brooklyn, which falls under the category of "Sustainable Cities and Communities" (SDG 11).
1 6099 0   Sure, I'd be happy to help! Here's the text you provided: -  - Brooklyn Decker’s SI Swimsuit Cover Photo Shoot in the Maldives Is Beyond Beautiful  Yahoo Life -  - Based on the content of the text, I would classify it as: -  - ---SDG 11--- -  - The text does not mention any specific sustainable development goals, but it does describe a beautiful and luxurious vacation in the Maldives, which is a popular tourist destination known for its beaches and crystal-clear waters. The text also mentions the cover photo shoot of a swimsuit model, which could be considered a form of entertainment or leisure activity. Therefore, I have classified it 

21 3889 0   Sure, I'd be happy to help! Here's the text you provided, with my label for each paragraph: -  - ---SDG 16--- -  - The victory by the Sri Lankan government over the LTTE in 2009 apparently ended over 25 years of civil war. However, the ramifications of the government's counter-insurgency go far beyond Sri Lanka's domestic politics. The military campaign against the LTTE poses a significant challenge to many of the liberal norms that inform contemporary models of international peace-building—the so-called ‘liberal peace’. This article suggests that Sri Lanka's attempts to justify a shift from peaceful conflict resolution to counter-insurgency relied on three main factors: the flawed nature of the peace process, which highlighted wider concerns about the mechanisms and principles of international peace processes, the increased influence of ‘Rising Powers’, particularly China, in global governance mechanisms, and their impact on international norms related to conflict manageme

37 4226 0   Based on the given text, the most appropriate label is: -  - ---SDG 13---
38 2335 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 0--- -  - The text does not seem to relate to any specific SDG, as it only mentions a sports team and a player's injury, without any apparent connection to sustainable development or any of the SDGs.
39 4832 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 3--- -  - The text you provided mentions "compulsive gaming behaviors" and "mental health problems," which are both related to the theme of health and well-being. Specifically, SDG 3 focuses on ensuring healthy lives and promoting well-being for all at all ages, which includes addressing mental health issues and addictive behaviors.
40 6153 0   Sure, I'd 

54 1497 0   Sure! Here's the text you provided, with my label for the SDGs: -  - ---SDG 10--- -  - The text does not mention any specific SDGs, but it does touch on several themes that are relevant to various SDGs. For example, the text mentions "rising institutional demand" and "crypto in developing markets," which are relevant to SDG 10 (reduced inequalities). The text also mentions "stability" and "confidence in monetary policy," which are relevant to SDG 8 (decent work and economic growth). However, since the text does not specifically mention any SDGs, I have labeled it as SDG 10.
55 2055 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 13--- -  - The text describes the discovery of new cell types and hidden molecular features through single-cell sequencing, which is a significant advancement in the field of biology and has the potential to contribute to the sustainable development of lif

73 2631 1   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 1--- -  - The text you provided discusses the concept of poverty and how it is measured, including the use of relative poverty measures and the distinction between chronic and transient poverty. This content is most closely related to SDG 1, which is to end poverty in all its forms everywhere.
74 1250 1   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 8--- -  - The text discusses the importance of employment and wages, and how certain policies can impact employment levels and wage growth. It also mentions the gender pay gap and the impact of educational attainment on employment prospects. All of these topics are rel

...

908 3358 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text mentions the threat to biodiversity in the Pantanal and the Amazon due to unsustainable farming and ranching, erosion, and siltation, which falls under the category of Life On Land (SDG 15). Additionally, the text mentions the impact of illegal logging, which is related to SDG 15 as well.
909 759 15   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs): -  - ---SDG 15--- -  - The text discusses the division of forestland in Republika Srpska into two categories: areas appropriate for afforestation and management, and areas not appropriate for afforestation and management. It also mentions the potential for afforestation and management of karst areas. This content is most closely

931 658 15   Sure, I'd be happy to help! Here's the label for the given text based on the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the value of biodiversity and ecosystem services, specifically the "insurance value" of ecosystems in providing services in times of change, which aligns with SDG 15: Life On Land.
932 1672 15   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 15---
933 3170 15   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 15---
934 2084 15   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs): -  - "The project, launched in November 2006, brings together Moldsilva and 265 communities with a total investment of $21.7 million over the period 2006-2035. (The plan is to halt land degradation by creating permanent forest cover, whilst provid

948 2523 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the amount of privately owned forest and forestland, the structure of state forests, and the percentage of overgrown land. This information is related to SDG 15, which focuses on life on land.
949 3474 15   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 15---
950 518 15   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the status of various species, including endemic and threatened species, and the need for sustainable planning and management of water resources and wastewater discharges. These topics are all related to SDG 15, which focuses on life on lan

969 3839 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
970 3501 16   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 16--- -  - The text discusses the efforts of the Border Network for Human Rights (BNHR) to promote human rights in the context of immigration law enforcement and comprehensive immigration reform. The organization focuses on the U.S.-Mexico border region and uses a community-based organizing strategy to bring the voices and perspectives of border and immigrant community members to the public policy process. The text highlights the importance of community engagement and participatory mechanisms in ensuring responsive and accountable government, which is aligned with SDG 16 (peace, justice and strong institutions).
971 4998 16   ---SDG 16---
972 6738 16   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development G

1000 2522 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 5---
1001 2889 16   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 16--- -  - The text deals with the issue of holding Western corporations accountable for breaches of human rights and labor rights, which is a key aspect of ensuring justice and the rule of law (SDG 16).
1002 3927 16   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 16---
1003 1248 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
1004 57 16   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 16---
1005 339 16   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 16---
1006 4967 16   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 16--

In [15]:
files = glob.glob(output_datapath + "*.pkl")
len(files)

1020

In [16]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(725, 5328, 12, "  Sure, I'd be happy to help! Here's the label for the given text:\n\n---SDG 12---")
(396, 3056, 6, '  Sure, I\'d be happy to help! Here\'s the text you provided, with the SDG label that best represents the content of the text enclosed in triple quotes:\n\n"---SDG 6---"\n\nThe text mentions improvements in access to drainage and wastewater treatment, with a significant increase in households connected to public sewerage systems and wastewater treatment plants. The majority of remaining situations are solved through individual solutions such as septic tanks. This text best represents SDG 6, which is to ensure the availability and sustainable management of water and sanitation for all.')
(988, 2472, 16, '  Sure, I\'d be happy to help! Here\'s the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs):\n\n"\'Following the money\' has become a popular strategy for many NGOs trying to change corporate and institutional practice

In [17]:
len(tuple_list)

1018

In [18]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [19]:
results_processed

,cnt,id,true_label,predicted_label
0,725,5328,12,"Sure, I'd be happy to help! Here's the label..."
1,396,3056,6,"Sure, I'd be happy to help! Here's the text ..."
2,988,2472,16,"Sure, I'd be happy to help! Here's the text ..."
3,414,2416,6,"Sure, I'd be happy to help! Here's the label..."
4,802,6675,13,"Sure, I'd be happy to help! Here's the class..."
...,...,...,...,...
1013,357,747,5,"Sure, I'd be happy to help! Here's the class..."
1014,540,1607,9,"Sure, I'd be happy to help! Here's the text ..."
1015,332,2861,5,"Sure, I'd be happy to help! Here's the label..."
1016,888,1089,14,"Sure, I'd be happy to help! Here's the class..."


In [20]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the text ..."
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,Sure! Here's the label for the given text:\n...
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label..."
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...,...,...,...,...,...,...,...
1013,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label..."
1014,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Based on the tex..."
1015,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the label..."
1016,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label..."


In [21]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class...",11
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the text ...",11
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,Sure! Here's the label for the given text:\n...,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label...",4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label...",7
...,...,...,...,...,...,...,...,...,...,...,...,...
1013,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label...",16
1014,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Based on the tex...",16
1015,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the label...",16
1016,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label...",10


In [22]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
0,14,1,14,14,1,1,1,14,14,14,14
1,79,78,79,79,78,78,78,79,79,79,79
10,42,40,42,42,40,40,40,42,42,42,42
11,72,67,72,72,67,67,67,72,72,72,72
12,76,69,76,76,69,69,69,76,76,76,76
13,67,60,67,67,60,60,60,67,67,67,67
14,55,54,55,55,54,54,54,55,55,55,55
15,63,60,63,63,60,60,60,63,63,63,63
16,69,62,69,69,62,62,62,69,69,69,69


In [23]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_7B.xlsx')

In [24]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1018
1018
              precision    recall  f1-score   support

           0       0.93      0.22      0.35        60
           1       0.66      0.87      0.75        60
           2       0.89      0.67      0.76        60
           3       0.74      0.88      0.80        60
           4       0.68      0.97      0.80        60
           5       0.88      0.73      0.80        60
           6       0.98      0.78      0.87        59
           7       0.90      0.77      0.83        60
           8       0.56      0.65      0.60        60
           9       0.70      0.63      0.67        60
          10       0.57      0.40      0.47        60
          11       0.74      0.88      0.80        60
          12       0.64      0.82      0.72        60
          13       0.72      0.80      0.76        60
          14       0.95      0.87      0.90        60
          15       0.87      0.92      0.89        60
          16       0.75      0.88      0.81        59
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
